# Accessing Data

In [6]:
#This gets us into the right directory from home, in order to run the import python script from Mat

#sys = system (module), gives information and control over python interpreter/terminal itself
import sys
sys.path.append("/home/565/pv3484/aus_substation_electricity")

#% is a magic command, special shortcut command that lets you control/interact with notebook environment (ie. gives control of terminal without writing full python code)
%cd aus_substation_electricity/

!pwd

#This section imports the substations that Mat put together

%run /home/565/pv3484/aus_substation_electricity/import_substation.py

[Errno 2] No such file or directory: 'aus_substation_electricity/'
/home/565/pv3484/aus_substation_electricity
/home/565/pv3484/aus_substation_electricity
processing nsw substations for ['ausgrid'] from None to None
ausgrid
following columns in demand are not in info index:
['MT_HU', 'SI_NO']
removing these columns from demand
number of substations in ausgrid substation info: 134
number of substations in ausgrid substation data: 132
following sites match selection criteria:
               energy_asset          Name  Area  Dwellings  Persons  Residential  Commercial  Industrial  Primary Production  Education  \
ID                                                                                                                                        
BLAKE         AG_BLAKEHURST    Blakehurst     7      10081    28521        0.850       0.005       0.021               0.000      0.022   
PUNCH          AG_PUNCHBOWL     Punchbowl     9      17514    50395        0.826       0.048       0.038

In [19]:
# Define the all national public holidays (including moving i.e. easter, is accounted for)
holidays = {
    "New Year's Day": lambda y: pd.Timestamp(f"{y}-01-01"),
    "Australia Day": lambda y: pd.Timestamp(f"{y}-01-26"),
    "Good Friday": lambda y: easter(y) - pd.Timedelta(days=2),
    "Easter Saturday": lambda y: easter(y) - pd.Timedelta(days=1),
    "Easter Sunday": lambda y: easter(y),
    "Easter Monday": lambda y: easter(y) + pd.Timedelta(days=1),
    "ANZAC Day": lambda y: pd.Timestamp(f"{y}-04-25"),
    "Christmas Day": lambda y: pd.Timestamp(f"{y}-12-25"),
    "Boxing Day": lambda y: pd.Timestamp(f"{y}-12-26"),
}

# Temperature Observations
- creating a function so I can plot any day and year range temperature obsersation for substations

In [41]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
from dateutil.easter import easter

# Define all national public holidays (fixed + moving)
holidays = {
    "New Year's Day": lambda y: pd.Timestamp(f"{y}-01-01"),
    "Australia Day": lambda y: pd.Timestamp(f"{y}-01-26"),
    "Good Friday": lambda y: pd.Timestamp(easter(y)) - pd.Timedelta(days=2),
    "Easter Saturday": lambda y: pd.Timestamp(easter(y)) - pd.Timedelta(days=1),
    "Easter Sunday": lambda y: pd.Timestamp(easter(y)),
    "Easter Monday": lambda y: pd.Timestamp(easter(y)) + pd.Timedelta(days=1),
    "ANZAC Day": lambda y: pd.Timestamp(f"{y}-04-25"),
    "Christmas Day": lambda y: pd.Timestamp(f"{y}-12-25"),
    "Boxing Day": lambda y: pd.Timestamp(f"{y}-12-26"),
}

def plot_holiday_temperatures(obs, holidays, start_year=2004, end_year=2018,
                              base_dir="/home/565/pv3484/aus_substation_electricity/figures/temperature_obs"):
    """
    Plot temperature observations for selected columns across multiple years,
    saving each holiday's plots into a single multi-page PDF file in base_dir.
    """
    # Ensure datetime index
    obs.index = pd.to_datetime(obs.index)
    
    # Keep your rename_map exactly as you wrote it
    rename_map = {
        "t2m": "Average Temperature",
        "t2m_30max": "Maximum Temperature",
        "t2m_30min": "Minimum Temperature"
    }
    obs = obs.rename(columns=rename_map)
    
    # Only keep the renamed temperature columns
    temp_columns = list(rename_map.values())
    
    # Define viridis colour palette for years
    years = list(range(start_year, end_year+1))
    palette = sns.color_palette("Dark2", len(years))
    
    # Ensure base_dir exists
    os.makedirs(base_dir, exist_ok=True)
    
    for holiday_name, holiday_func in holidays.items():
        pdf_path = os.path.join(base_dir, f"{holiday_name.replace(' ', '_')}.pdf")
        
        with PdfPages(pdf_path) as pdf:
            for temp_col in temp_columns:
                plt.figure(figsize=(10, 4))
                
                for i, year in enumerate(years):
                    holiday_date = pd.Timestamp(holiday_func(year))
                    day_data = obs[obs.index.date == holiday_date.date()].dropna(subset=[temp_col])
                    
                    if not day_data.empty:
                        hour_offset = day_data.index.hour + day_data.index.minute / 60
                        plt.plot(hour_offset, day_data[temp_col],
                                 label=str(year),
                                 color=palette[i])
                    else:
                        plt.plot([], [], label=f"{year} (no data)", color=palette[i], linestyle=":")
                
                plt.title(f'{holiday_name} – {temp_col} ({start_year}-{end_year})', fontsize=16)
                plt.xlabel('Hour of Day', fontsize=14)
                plt.ylabel('Temperature Observation', fontsize=14)
                plt.xticks(range(0, 25, 2))
                plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10)
                plt.tight_layout()
                
                # Save this figure as a new page in the holiday PDF
                pdf.savefig()
                plt.close()
        
        print(f"✅ Saved multi-page PDF for {holiday_name}: {pdf_path}")

## Using the function above

In [42]:
plot_holiday_temperatures(obs, holidays, start_year=2004, end_year=2018)

✅ Saved multi-page PDF for New Year's Day: /home/565/pv3484/aus_substation_electricity/figures/temperature_obs/New_Year's_Day.pdf
✅ Saved multi-page PDF for Australia Day: /home/565/pv3484/aus_substation_electricity/figures/temperature_obs/Australia_Day.pdf
✅ Saved multi-page PDF for Good Friday: /home/565/pv3484/aus_substation_electricity/figures/temperature_obs/Good_Friday.pdf
✅ Saved multi-page PDF for Easter Saturday: /home/565/pv3484/aus_substation_electricity/figures/temperature_obs/Easter_Saturday.pdf
✅ Saved multi-page PDF for Easter Sunday: /home/565/pv3484/aus_substation_electricity/figures/temperature_obs/Easter_Sunday.pdf
✅ Saved multi-page PDF for Easter Monday: /home/565/pv3484/aus_substation_electricity/figures/temperature_obs/Easter_Monday.pdf
✅ Saved multi-page PDF for ANZAC Day: /home/565/pv3484/aus_substation_electricity/figures/temperature_obs/ANZAC_Day.pdf
✅ Saved multi-page PDF for Christmas Day: /home/565/pv3484/aus_substation_electricity/figures/temperature_obs/